# MSiA 400 - Final Project
## Feature engineering for transaction
### Team 12: Alejandra Lelo de Larrea Ibarra, Xin Shu, Yi Chen, Yiqing Cheng

In [3]:
import pandas as pd
import psycopg2
from psycopg2 import Error
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="user",
                                  password="password",
                                  host="pg.analytics.northwestern.edu",
                                  port="5432",
                                  database="2022-everything-team12")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
df_trnsact = pd.read_sql("select * from \"TRNSACT\"order by random() LIMIT 1000000", connection);
# df_trnsact = pd.read_sql("select * from \"TRNSACT\"", connection);
if (connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed") 


You are connected to -  ('PostgreSQL 10.12 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-39), 64-bit',) 

PostgreSQL connection is closed


In [5]:
def create_time_features(df):
    df = df.copy()
    df['quarter'] = pd.PeriodIndex(df_trnsact['SALEDATE'], freq='Q')
    df['month'] = pd.PeriodIndex(df_trnsact['SALEDATE'], freq='M')
    df['year'] = pd.PeriodIndex(df_trnsact['SALEDATE'], freq='Y')
    df['weekofyear'] = pd.PeriodIndex(df_trnsact['SALEDATE'], freq='D')
    return df

df_trnsact.SALEDATE = pd.to_datetime(df_trnsact.SALEDATE)
df = create_time_features(df_trnsact)

df = pd.get_dummies(df, prefix=['STYPE'], columns=['STYPE'])
df_groupyed = df.groupby(['STORE', 'month']).agg({'SKU': 'count', 'STYPE_P': 'count','STYPE_R':'count','AMT':'sum' })
df_groupyed

df_groupyed = df.groupby(['STORE', 'month']).agg({'SKU': 'count', 'STYPE_P': 'count','STYPE_R':'count','AMT':'sum' })
df_groupyed['AMT_1'] = df_groupyed['AMT'].shift(1)
df_groupyed['AMT_3'] = df_groupyed['AMT'].shift(3)
df_groupyed['AMT_6'] = df_groupyed['AMT'].shift(6)
df_groupyed['AMT_9'] = df_groupyed['AMT'].shift(9)
df_groupyed





SKU  STYPE_P  STYPE_R     AMT   AMT_1   AMT_3   AMT_6   AMT_9
STORE month                                                                 
102   2004-08    4        4        4  108.25     NaN     NaN     NaN     NaN
      2004-09    2        2        2    7.48  108.25     NaN     NaN     NaN
      2004-10    3        3        3   89.00    7.48     NaN     NaN     NaN
      2004-11    6        6        6  185.95   89.00  108.25     NaN     NaN
      2004-12    3        3        3  113.23  185.95    7.48     NaN     NaN
...            ...      ...      ...     ...     ...     ...     ...     ...
9909  2005-02    2        2        2   22.25   12.50   57.98  205.99   52.00
      2005-03    1        1        1   22.00   22.25   54.00    6.12  110.99
      2005-04    5        5        5  125.50   22.00   12.50   65.00  166.74
      2005-06    3        3        3  109.49  125.50   22.25   57.98  205.99
      2005-08    1        1        1    1.99  109.49   22.00   54.00    6.12

[3627 rows x 8 columns]